In [ ]:
import yaml
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

conf = yaml.safe_load(open("vis_config.yml"))

img_dir = Path(conf["paths"]["image_dir"]).resolve()
results_dir = Path(conf["paths"]["results_dir"]).resolve()
csv_results_dir = Path(conf["paths"]["csv_results_dir"]).resolve()

image_conf = conf['image']
input_shape = (image_conf["height"], image_conf["width"], image_conf["channels"])


In [ ]:
import pandas as pd

eval_header = ['Model', 'EvalMethod', "EvalParam", 'AtribMethod', 'n', 'Result']

df = pd.read_csv(csv_results_dir / "resnet_vgg_output_file.csv")
df_train = pd.read_csv(csv_results_dir / "train_comp_output_file.csv")
df_lrponly = pd.read_csv(csv_results_dir / "resnet_lrponly_output_file.csv")
#df_rand = pd.read_csv(csv_results_dir / "rand_output_file.csv")
df_mobile = pd.read_csv(csv_results_dir / "mobilenet_output_file.csv")
df_selectivity = pd.read_csv(csv_results_dir / "selectivity_output_file.csv")
df_sens = pd.read_csv(csv_results_dir / "sens_output_file.csv")


#df = df[df.EvalMethod != "LimeWrapper(Rule.IDENTITY_RULE)"].append(df_lime)
#df = df[df.AtribMethod != "RandomAttribution(None)"].append(df_rand)

df = pd.concat([df,df_mobile])
df = df.query("not (EvalMethod=='RemoveBestPixel' & AtribMethod=='LimeWrapper(Rule.IDENTITY_RULE)')"
    ).append(df_selectivity.query("EvalMethod=='RemoveBestPixel'"))
df = df.query("not (EvalMethod=='BlurBestPixel' & AtribMethod=='LimeWrapper(Rule.IDENTITY_RULE)')"
    ).append(df_selectivity.query("EvalMethod=='BlurBestPixel'"))
    
df = df.query("not (EvalMethod=='SensitivityN')"
    ).append(df_sens.query("EvalMethod=='SensitivityN'"))

#df = df.query("Model=='vgg19' | Model=='resnet50' | Model=='mobilenet'")
#df = df.query("Model=='custom_custom_model_resnet50_02' | Model=='custom_custom_model_resnet50_04' | Model=='custom_custom_model_resnet50_06' | Model=='custom_custom_model_resnet50_08' | Model=='custom_custom_model_resnet50_10'")

df = pd.concat([df,df_train])

del df['EvalParam']
df = df[df.AtribMethod != "RandomAttribution(None)"]
df=df.reset_index()


markers = {
    "GradientBased(Rule.GRAD_ONLY)": ".",
    "GradientBased(Rule.GRAD_X_INPUT)": "s",
    "GradientBased(Rule.INTEGRATED_GRAD)": "X",
    "GradientBased(Rule.RANDOM_BASELINE_INTEGRATED_GRAD)": "D",
    "LRP(Rule.Z_RULE)": "P",
    "LimeWrapper(Rule.IDENTITY_RULE)": "*",
    "RandomAttribution(None)": "|",
#}
#markers = {
    "LRP(Rule.HEURISTIC_RULE)": ".",
    "LRP(Rule.OMEGA_RULE)": "s",
    "LRP(Rule.IDENTITY_RULE)": "X",
    "LRP(Rule.EPSILON_RULE)": "D",
    "LRP(Rule.Z_RULE)": "P"
}
df.head()

# Plots


## SensN

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.EvalMethod == "SensitivityN"]



model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, marker=markers[str(name[1])],markersize=5, label=str(name[1]))

    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.8),fontsize=20)
fig.show()


## RemoveBestPixel

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.EvalMethod == "RemoveBestPixel"]
df_filtered.EvalMethod = "Selectivity(Zeros,Squares)"

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, marker=markers[str(name[1])],markersize=10, label=str(name[1]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-1),fontsize=20)
fig.show()

## RemoveBestSuperixel

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.EvalMethod == "RemoveBestSuperpixel"]
df_filtered.EvalMethod = "Selectivity(Zeros,Superpixels)"

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, marker=markers[str(name[1])],markersize=10, label=str(name[1]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()

## BlurBestPixel

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.EvalMethod == "BlurBestPixel"]
df_filtered.EvalMethod = "Selectivity(Mean,Squares)"

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, marker=markers[str(name[1])],markersize=10, label=str(name[1]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()


## BlurBestSuperpixel

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.EvalMethod == "BlurBestSuperpixel"]
df_filtered.EvalMethod = "Selectivity(Mean,Superpixels)"

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, marker=markers[str(name[1])],markersize=10, label=str(name[1]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()


## IROF

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.EvalMethod == "IROF_Remove"]
df_filtered.EvalMethod = "IROF(Zeros)"
#df_filtered.Result =  df_filtered.n  -df_filtered.Result + 1

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))

fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, marker=markers[str(name[1])],markersize=10, label=str(name[1]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()


In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.EvalMethod == "IROF_Blur"]
df_filtered.EvalMethod = "IROF(Mean)"

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, marker=markers[str(name[1])],markersize=10, label=str(name[1]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()


## Lime

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.AtribMethod.str.contains("Lime")]

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, label=str(name[0]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()



## LRP

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.AtribMethod.str.contains("LRP")]

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(20)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, label=str(name[0]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()



## GradBased

In [ ]:
df_filtered = df
df_filtered = df_filtered[df_filtered.AtribMethod.str.contains("GRAD")]

model_grouped = df_filtered.groupby('Model')

fig, axs = plt.subplots(len(model_grouped))
fig.set_figheight(30)
fig.set_figwidth(30)
for i, (model_name, model_g) in enumerate(model_grouped):
    axs[i].set_title(str(model_name))
    for name, g in model_g.groupby(['EvalMethod', 'AtribMethod']):
        axs[i].plot(g.n, g.Result, label=str(name[0]))
    axs[i].axhline(color='black')
    axs[i].set_xlabel("Number of Iterations")
    axs[i].set_ylabel("Evaluation Score")
axs[-1].legend(loc=(0,-0.9),fontsize=20)
fig.show()


# Anova

### One-way


In [ ]:
new_df = df[df.AtribMethod != "RandomAttribution(None)"]

df_list = []
for eval_method in list(new_df.EvalMethod.unique()):
    temp = new_df[new_df.EvalMethod == eval_method]
    temp_idx = temp.groupby(['Model', 'EvalMethod', 'AtribMethod'])['n'].idxmax()
    
    df_list.append(temp.loc[temp_idx])
new_df = pd.concat(df_list, ignore_index=True)
new_df


In [ ]:
import researchpy as rp

rp.summary_cat(new_df[['Model', 'EvalMethod', 'AtribMethod']])

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

temp_df = new_df

query_str = '&'.join([
    f"Model!='{m}'" 
    for m in [
        'vgg19', 'resnet50', 'mobilenet'
        ]])
temp_df = temp_df.query(query_str)

print("All")

model = ols('FinalScore ~ C(Model)', data=temp_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
display(anova_table)

for q in temp_df.EvalMethod.unique():
    print(q)
    temp_df_part = temp_df.query(f"EvalMethod=='{q}'")

    model = ols('FinalScore ~ C(Model)', data=temp_df_part).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    display(anova_table)

In [ ]:
temp_df = new_df


query_str = '|'.join([
    f"Model=='{m}'" 
    for m in [
        'vgg19', 'resnet50', 'mobilenet'
        ]])
temp_df = temp_df.query(query_str)

print("All")
model = ols('FinalScore ~ C(Model)', data=temp_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
display(anova_table)

for q in temp_df.EvalMethod.unique():
    print(q)
    temp_df_part = temp_df.query(f"EvalMethod=='{q}'")

    model = ols('FinalScore ~ C(Model)', data=temp_df_part).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    display(anova_table)

In [ ]:
temp_df = new_df

pd.options.display.float_format = '{:,.5f}'.format



temp_df = temp_df.replace(['custom_custom_model_resnet50_02'],'Custom ResNet50 (2 epoch)')
temp_df = temp_df.replace(['custom_custom_model_resnet50_04'],'Custom ResNet50 (4 epoch)')
temp_df = temp_df.replace(['custom_custom_model_resnet50_06'],'Custom ResNet50 (6 epoch)')
temp_df = temp_df.replace(['GradientBased(Rule.GRAD_ONLY)'],'Gradients Only')
temp_df = temp_df.replace(['GradientBased(Rule.GRAD_X_INPUT)'],'Gradient x Input')
temp_df = temp_df.replace(['GradientBased(Rule.INTEGRATED_GRAD)'],'Integrated Gradients')
temp_df = temp_df.replace(['GradientBased(Rule.RANDOM_BASELINE_INTEGRATED_GRAD)'],'Random Baseline Integrated Gradients')
temp_df = temp_df.replace(['LRP(Rule.Z_RULE)'],'LRP(Z Rule)')
temp_df = temp_df.replace(['LimeWrapper(Rule.IDENTITY_RULE)'], 'LIME')
temp_df = temp_df.replace(['SensitivityN'], 'Sensitivity-N')
temp_df = temp_df.replace(['IROF_Remove'], 'IROF(Zeros)')
temp_df = temp_df.replace(['IROF_Blur'], 'IROF(Mean)')
temp_df = temp_df.replace(['RemoveBestPixel'], 'Selectivity(Zeros, Squares)')
temp_df = temp_df.replace(['BlurBestPixel'], 'Selectivity(Mean, Squares)')
temp_df = temp_df.replace(['RemoveBestSuperpixel'], 'Selectivity(Zeros, Superpixels)')
temp_df = temp_df.replace(['BlurBestSuperpixel'], 'Selectivity(Mean, Superpixels)')

temp_df = temp_df.pivot_table(
        values='FinalScore', 
        index=['Model', 'AtribMethod'], 
        columns='EvalMethod',
        sort=False)


temp_df

### Tukey

In [ ]:

from statsmodels.stats.multicomp import pairwise_tukeyhsd

temp_df = new_df

query_str = '|'.join([
    f"Model=='{m}'" 
    for m in [
        'vgg19', 'resnet50', 'mobilenet'
        ]])
temp_df = temp_df.query(query_str)

print("All")
temp_df_part = temp_df.query(f"EvalMethod=='{q}'")
tukey = pairwise_tukeyhsd(endog=temp_df['FinalScore'],
                        groups=temp_df['Model'],
                        alpha=0.05)

tukey.plot_simultaneous()

display(tukey.summary())


for q in temp_df.EvalMethod.unique():
    print(q)
    temp_df_part = temp_df.query(f"EvalMethod=='{q}'")
    tukey = pairwise_tukeyhsd(endog=temp_df_part['FinalScore'],
                            groups=temp_df_part['Model'],
                            alpha=0.05)

    tukey.plot_simultaneous()

    display(tukey.summary())

In [ ]:

from statsmodels.stats.multicomp import pairwise_tukeyhsd

temp_df = new_df

query_str = '&'.join([
    f"Model!='{m}'" 
    for m in [
        'vgg19', 'resnet50', 'mobilenet'
        ]])
temp_df = temp_df.query(query_str)

print("All")
tukey = pairwise_tukeyhsd(endog=temp_df['FinalScore'],
                        groups=temp_df['Model'],
                        alpha=0.05)

tukey.plot_simultaneous()

display(tukey.summary())


for q in temp_df.EvalMethod.unique():
    print(q)
    temp_df_part = temp_df.query(f"EvalMethod=='{q}'")
    tukey = pairwise_tukeyhsd(endog=temp_df_part['FinalScore'],
                            groups=temp_df_part['Model'],
                            alpha=0.05)

    tukey.plot_simultaneous()

    display(tukey.summary())

In [ ]:
temp_df = new_df
temp_df.boxplot(column=["FinalScore"], by=["Model"], figsize=(15,20))